In [1]:
from z3 import *
import math

## Utils

In [2]:
#TODO: sposta gli utils tutti qua su
def flatten(matrix):
    return [e for row in matrix for e in row]

## Encodings

#### At most/least one & exactly one

In [3]:
def at_least_one_seq(bool_vars):
    return Or(bool_vars)

def at_most_one_seq(x, name):
    # bool_vars renamed to x for simplicity
    n = len(x)
    if n == 1:
        return True
    s = [Bool(f"s_{i}_{name}") for i in range(n-1)]     # "name" is used in order to create a
                                                        # unique set of variables s for each constraint
                                                        # of the kind at_most_one_seq, so that they do not overlap 
                                                        # for different constraints!
                                                        # N.B. len(s) = n-1 != n = len (x)
    clauses = []
    clauses.append(Or(Not(x[0]), s[0]))                 # x[0] -> s[0] (s[i] modeled as: s[i] is true if the SUM UP TO index i IS 1!) i.e. x_1 -> s_1 in math notation
    for i in range(1, n-1):
        clauses.append(Or(Not(x[i]), s[i]))             # these two clauses model (x[i] v s[i-1]) -> s[i]
        clauses.append(Or(Not(s[i-1]), s[i]))
        clauses.append(Or(Not(s[i-1]), Not(x[i])))      # this one models s[i-1] -> not x[i]
    clauses.append(Or(Not(s[-1]), Not(x[-1])))          # s[n-2] -> not x[n-1]  i.e. s_(n-1) -> s_n in the mathematical notation (1-based)
    return And(clauses)

def exactly_one_seq(bool_vars, name):
    return And(at_least_one_seq(bool_vars), at_most_one_seq(bool_vars, name))

#### Conversion from int to bin

In [46]:
def num_bits(x):
    return math.floor(math.log2(x)) + 1

def find_max_digits(s, l): # TODO Idea: optimize number of digits depending on cases
    return num_bits(max(sum(s), max(l)))

In [47]:
for i in range(18):
    print(f"{i+1} --> {num_bits(i+1)}")

1 --> 1
2 --> 2
3 --> 2
4 --> 3
5 --> 3
6 --> 3
7 --> 3
8 --> 4
9 --> 4
10 --> 4
11 --> 4
12 --> 4
13 --> 4
14 --> 4
15 --> 4
16 --> 5
17 --> 5
18 --> 5


In [48]:
def int_to_bin(x, digits):  # TODO read: #https://ericpony.github.io/z3py-tutorial/guide-examples.htm
    x_bin = [(x%(2**(i+1)) // 2**i)==1 for i in range(digits-1,-1,-1)]
    return x_bin

def bin_to_int(x):
    n = len(x)
    x = sum(2**(n-i-1) for i in range(n) if x[i] == 1)
    return x

#### Operations on binary numbers

In [66]:
def equal(v, u, digits):
    return And([v[k]==u[k] for k in range(digits)])

def all_false(v):
    return And([Not(v[k]) for k in range(len(v))])

def leq(v, u, digits):
    """Encoding of v <= u

    Args:
        v (list[bool]): binary representation of v
        u (list[bool]): binary representation of u
        digits (int): number of digits of v and u

    Returns:
        Z3 formula: encoding of v <= u in binary considering their {digits} most significant bits
    """
    assert(len(v) == len(u) and len(u) == digits)
    if digits == 1:
        return Or(v[0]==u[0], And(Not(v[0]), u[0]))
    else:
        return Or(And(Not(v[0]), u[0]), 
                  And(v[0]==u[0], leq(v[1:], u[1:], digits-1)))


In [67]:
def sum_bin(a_bin, b_bin, d_bin, digits, name):
    """Encodes into a SAT formula the binary sum {a_bin + b_bin = d_bin}, each number having {digits} num of bits

    Args:
        a_bin (list[bool]): binary representation of a
        b_bin (list[bool]): binary representation of b
        d_bin (list[bool]): binary representation of d
        digits (int): number of bits of each number 
        name (str): string to identify carry boolean variables

    Returns:
        (Z3-expression): formula representing SAT encoding of binary sum
    """

    #slide SAT-I
    c = [Bool(f"c_{k}_{name}") for k in range(digits+1)]
    c[0] = False
    c[-1] = False

    clauses = []
    for k in range(digits-1,-1,-1):
        # clauses.append((a_bin[k] == b_bin[k]) == (c[k+1] == d_bin[k]))    #TODO: check if it is faster
        clauses.append(d_bin[k] == Or(And(a_bin[k], b_bin[k], c[k+1]), And(a_bin[k], Not(b_bin[k]), Not(c[k+1])),
                                   And(Not(a_bin[k]), b_bin[k], Not(c[k+1])), And(Not(a_bin[k]), Not(b_bin[k]), c[k+1])))
        clauses.append(c[k] == Or(And(a_bin[k],b_bin[k]), And(a_bin[k],c[k+1]), And(b_bin[k],c[k+1])))

    return And(clauses)


def conditional_sum_K_bin(x, alpha, delta, name):
    """Encodes into a SAT formula the constraint {delta = sum_over_j(alpha[j] | x[j] == True)}

    Args: 
        x (list[Bool]): list of Z3 Variables, i.e. x_j tells wether or not to add alpha_j to the sum
        alpha (list[list[bool]]): list of known coefficients, each one represented as list[bool] i.e. binary number, whose subset will be summed in the constraint
        delta (list[Bool]): list of Z3 Variables, which will be constrained to represent the sum
        name (string): to uniquely identify the created variables
    Returns:
        formula (Z3-expression): And of clauses representing SAT encoding of Linear Integer constraint
    
    """
    n = len(x)
    digits = len(delta)

    # matrix containing temporary results of sum_bin
    d = [[Bool(f"d_{j}_{k}_{name}") for k in range(digits)] for j in range(n-1)]    # j = 1..n-1 because last row will be delta
    d.append(delta)

    clauses = []

    # row 0
    clauses.append( And(Implies(x[0], equal(d[0], alpha[0], digits)),              # If x[0] == 1 then d_0 == alpha_0
                        Implies(Not(x[0]), all_false(d[0]))))              # elif x[0] == 0 then d_0 == [0..0]

    # row j>1
    for j in range(1,n):
        clauses.append( And(Implies(x[j], sum_bin(d[j-1], alpha[j], d[j], digits, f"{name}_{j-1}_{j}")),           # If c_j == 1 then d_j == d_j-1 + alpha_j 
                            Implies(Not(x[j]), equal(d[j], d[j-1], digits))))
        
    return And(clauses)

#### Linear Integer constraint over binary numbers

In [68]:
def LinearInteger_bin(x, alpha, beta, name):
    """Encodes into a SAT formula the linear constraint {sum_over_j(alpha[j] | x[j] == True) <= beta}

    Args: 
        x (list[Bool]): list of Z3 Variables, i.e. x_j tells wether or not to add alpha_j to the sum
        alpha (list[list[bool]]): list of known coefficients, each one represented as list[bool] i.e. binary number, whose subset will be summed in the constraint
        beta (list[bool]): binary representing the known term, i.e. the upper bound for the sum
        name (string): to uniquely identify the created variables
    Returns:
        formula (Z3-expression): And of clauses representing SAT encoding of Linear Integer constraint
    """
    n = len(x)
    digits = len(beta)

    clauses = []

    ### TODO: remove if implementation of conditional_sum_K_bin works
    # matrix containing temporary results of sum_bin
    # d = [[Bool(f"d_{j}_{k}_{name}") for k in range(digits)] for j in range(n)]

    # clauses = []

    # # row 0
    # clauses.append( And(Implies(x[0], equal(d[0], alpha[0], digits)),              # If x[0] == 1 then d_0 == alpha_0
    #                     Implies(Not(x[0]), all_false(d[0]))))              # elif x[0] == 0 then d_0 == [0..0]
    
    # # row j>1
    # for j in range(1,n):
    #     clauses.append( And(Implies(x[j], sum_bin(d[j-1], alpha[j], d[j], digits, f"{name}_{j-1}_{j}")),           # If c_j == 1 then d_j == d_j-1 + alpha_j 
    #                         Implies(Not(x[j]), equal(d[j], d[j-1], digits))))                                      # elif c_j == 0 then d_j == d_j-1
    ### in favour of this:
    sum_result = [Bool(f"LIsum_result_{k}_{name}") for k in range(digits)]
    clauses.append(conditional_sum_K_bin(x, alpha, sum_result, name))
    ###


    # TODO: aggiungere a clause che d_n <= beta
    # I suppose to enter beta as integer (is it the best idea to optimize the program?)
    clauses.append(leq(sum_result, beta, digits)) 
    # TODO: does it make any sense to check d_j <= beta forall j or better to just do it for d_n?
    # TODO: have to check for overflows in num. digits?

    formula = And(clauses)
    return formula



In [69]:
# # testing of conditional_sum_K_bin
# n = 4
# digits = 6
# x = [Bool(f'x_{i}') for i in range(n)]
# delta = [Bool(f'delta_{j}') for j in range(digits)]
# alpha = [7, 9, 11, 4]
# alpha_bin = [int_to_bin(e, digits) for e in alpha]

# for comb in range(2**n):
#     s = Solver()
#     somma = 0
#     for i in range(n):
#         # print(comb, i, comb % (2**i))
#         if comb >= (2**i) and comb % (2**(i+1)) == 1:
#             print(comb, 2**i)
#             s.add(x[i])
#             somma += alpha[i]
#         else:
#             s.add(Not(x[i]))
#     s.add(conditional_sum_K_bin(x, alpha_bin, delta, 'ciao'))
#     s.check()
#     m = s.model()
#     print(f"{somma} ?= {bin_to_int([1 if m.evaluate(e) else 0 for e in delta])}")


# # print(s.check())
# # m = s.model()
# # print([m.evaluate(e) for e in x])
# # print([m.evaluate(e) for e in delta])

In [70]:
# # Testing of LinearInteger_bin
# n = 4
# digits = 6
# x = [Bool(f'x_{i}') for i in range(n)]
# alpha = [7, 9, 11, 6]
# alpha_bin = [int_to_bin(e, digits) for e in alpha]
# beta = int_to_bin(5, digits)
# s = Solver()
# s.add(LinearInteger_bin(x, alpha_bin, beta, 'ciao'))
# s.add(Or(x))

# print(s.check())
# m = s.model()
# print([m.evaluate(e) for e in x])


#### Functions to display solution

In [71]:
def interest_variables(model, a):
    return [[j for j in range(len(a[0])) if model.evaluate(a[i][j])] for i in range(len(a))]

def displayMCP(result):
    # TODO: fai piu bello per UNSAT
    if type(result) == str and result == "UNSAT":
        print(result)
    else:
        routes = result[0]
        obj_value = result[1]
        print(f"------Objective value: {obj_value}------")
        print(f"-------------Routes------------")
        m = len(routes)
        n = len(routes[0])
        for i in range(m):
            n = len(routes[i])
            v = n-1
            visited = []
            while True:
                visited.append(v)
                for k in range(n):
                    if routes[i][v][k] == True:
                        v = k
                        break
                if v == n-1:
                    break
            print('Origin --> ' + ' --> '.join([str(vertex) for vertex in visited[1:]]) + ' --> Origin')
            

#### Functions to display matrices of tensor and dist

In [72]:
### Add by Scaio

def tensor(model, bools):
    m = len(bools)
    n = len(bools[0])
    return [[[model.evaluate(bools[i][j][k]) for k in range(n)] for j in range(n)] for i in range(m)]

def display_matrices(tensor, name):
    m = len(tensor)
    n = len(tensor[0])
    for i in range(m):
        print(f'{name}_{i} matrix:')
        for j in range(n):
            print(tensor[i][j])
        print()

def courier_dist(model, r_i, a_i, D, n): # Implemented with the assumption where r_i is the matrix nxn where 
                                         # j is the object and k indicates, with respect to the other columns, the order of delivery
    dist = 0
    actual_pos = n #the origin, it would be n+1
    next_pos = n
    for k in range(n): #sum(a_i)+1):
        for j in range(n):
            if model.evaluate(r_i[k][j]):
                next_pos = j
        dist += D[actual_pos][next_pos]
        actual_pos = next_pos
    dist += D[actual_pos][n] #the origin, it would be n+1
    return dist

def max_dist(model, r, a, D, m, n):
    return max([courier_dist(model, r[i], a[i], D, n) for i in range(m)])

###

#### Function to check that routes are all Hamiltonian cycles

In [73]:
# TODO: funzioni per noi, da rimuovere alla
def check_all_hamiltonian(tensor):
    m = len(tensor)
    for i in range(m):
        if not check_hamiltonian(tensor[i]):
            return False
    return True

def check_hamiltonian(matrix):
    n = len(matrix)
    visited = [0] * n
    v = n-1
    while visited[v] == 0:
        visited[v] = 1
        for k in range(n):
            if matrix[v][k] == True:
                v = k
                break
    num_vertices = sum(row.count(True) for row in matrix)
    return (sum(visited) == num_vertices)


#### Objective function section

In [118]:
def distances_def_constraint(distances, flat_r, flat_D_bin):
    """Defines the set of binary numbers {distances} as the sum of the binary numbers in {flat_D_bin} whose respective element in {flat_r} is True
        i.e. distances[i] = sum_over_j(flat_D_bin[j] | (flat_r[i])[j] == True)  forall i

    Args:
        distances (list[list[Bool]]): distances[i] will be constrained to be the sum of flat_D_bin[j] with respective flat_r[i][j] == True
        flat_r (list[list[Bool]]]): list of flattened matrices of Z3 Bool variables, each matrix flat_r[i] originally (i.e. before flattening) describing the route of courier i
        flat_D_bin (list[list[bool]]): flattened matrix of distances D, with each element converted to a binary as a list[bool]
    
    Returns:
        (Z3 formula): encoding of the definition of distances
    """
    m = len(distances)

    clauses = []

    for i in range(m):
        clauses.append(conditional_sum_K_bin(flat_r[i], flat_D_bin, distances[i], "distances_def"))

    return And(clauses)    


def obj_function(model, distances):
    m = len(distances)
    maxdist = -1
    for i in range(m):
        dist = bin_to_int([1 if model.evaluate(distances[i][j]) else 0 for j in range(len(distances[i]))])
        maxdist = max(maxdist, dist)
    return maxdist

def AllLessEq_bin(distances, obj_value_bin, digits):
    m = len(distances)

    clauses = []

    for i in range(m):
        clauses.append(leq(distances[i], obj_value_bin, digits))

    return And(clauses)

## Assumptions

Assumpions that we can make:
1. $n >= m$
2. The distance matrix $D$ is quasimetric (more info [here](https://proofwiki.org/wiki/Definition:Quasimetric)) => Giving an item to courier who has none is always "same or better" than giving 2 to another courier and none to this one

(Note: 1. + 2. => can add implicit constraint "Every courier has at least one item assigned to it")

## Model 1

In [122]:
# TODO: add (and copy-implement) timeout decorator
def multiple_couriers_planning_1(m, n, l, s, D):
    """Model 1 in Z3 for the Multiple Couriers Planning problem

    Args:
        m (int): number of couriers
        n (int): number of items to deliver
        l (list[int]): l[i] represents the maximum load of courier i, for i = 1..m
        s (list[int]): s[j] represents the size of item j, for j = 1..n
        D (list[list[int]]): (n+1)x(n+1) matrix, with D[i][j] representing the distance from
                             distribution point i to distribution point j
                    
    """
    ## VARIABLES

    # a for assignments
    a = [[Bool(f"a_{i}_{j}") for j in range(n)] for i in range(m)]
    # a_ij = 1 indicates that courier i takes object j
    # O(m * n) vars

    # r for routes
    r = [[[Bool(f"r_{i}_{j}_{k}") for k in range(n+1)] for j in range(n+1)] for i in range(m)]  
    # r_ijk = 1 indicates that courier i moves from delivery point j to delivery point k in his route
    # n+1 delivery points because considering Origin point as well, representes as n+1-th row and column, for each courier i
    # O(m * n^2) vars

    solver = Solver()

    ## CONSTRAINTS
    # Conversions:
    digits = find_max_digits(s, l)
    s_bin = [int_to_bin(s_j, digits) for s_j in s]
    l_bin = [int_to_bin(l_i, digits) for l_i in l]
    print(f"digits = {digits}")


    # Constraint 1: every object is assigned to one and only one courier
    for j in range(n):
        solver.add(exactly_one_seq([a[i][j] for i in range(m)], f"assignment_{j}"))
    
    # Constraint 2: every courier can't exceed its load capacity
    for i in range(m):
        solver.add(LinearInteger_bin(a[i], s_bin, l_bin[i], f"load_capacity_courier_{i}"))

    # Constraint 3: every courier has at least 1 item to deliver (implicit constraint, because n >= m and distance is quasimetric (from discussion forum))
    # TODO: check if it actually speeds up execution
    for i in range(m):
        solver.add(at_least_one_seq(a[i]))

    #TODO: better to use "clauses" list than doing so many solver.add below? Or call entirely a function maybe

    # Constraint 4: routes
    # last row and last column represent the Origin point
    t = [[[Bool(f"courier_{i}_delivers_{j}_as_{k}-th") for k in range(n)] for j in range(n)] for i in range(m)] #TODO 3: move down, just for debugging

    for i in range(m):
        # Constraint 4.1: diagonal is full of zeros, i.e. can't leave from j to go to j
        solver.add(And([Not(r[i][j][j]) for j in range(n+1)]))

        # Constraint 4.2: row j has a 1 iff courier i delivers object j
        # rows
        for j in range(n):
            solver.add(Implies(a[i][j], exactly_one_seq(r[i][j], f"courier_{i}_leaves_{j}")))  # If a_ij then exactly_one(r_ij)
            solver.add(Implies(Not(a[i][j]), all_false(r[i][j])))   # else all_false(r_ij)
        solver.add(exactly_one_seq(r[i][n], f"courier_{i}_leaves_origin"))    # exactly_one in origin point row === courier i leaves from origin

        # Constraint 4.3: column j has a 1 iff courier i delivers object j
        # columns
        for k in range(n):
            solver.add(Implies(a[i][k], exactly_one_seq([r[i][j][k] for j in range(n+1)], f"courier_{i}_reaches_{k}")))  # If a_ij then exactly_one(r_i,:,k)
            solver.add(Implies(Not(a[i][k]), all_false([r[i][j][k] for j in range(n+1)])))   # else all_false(r_i,:,k) 
            # TODO imply false just for the one that weren't alredy counted in the rows (Edo: "Cos?")
        solver.add(exactly_one_seq([r[i][j][n] for j in range(n+1)], f"courier_{i}_returns_to_origin"))         # exactly_one in origin point column === courier i returns to origin
     
        # Constraint 4.4: HAMILTONIAN CYCLE
        # solver.add(Implies(all_false([r[i][j][k] for j in range(n+1)]), Not(z[k])))   #TODO: aggiungere perchè giusto, useless o implicit? Stessa domanda di sotto

        ### Edo's wrong idea:
        # # TODO: WRONG! Have to do zK == K-hops matrix IMO! Posso fare qui un set di K-hops matrices (m+n)x(m+n) e controllare tutti insieme!
        # # Altra idea: riuscire a encodarlo in una rappresentazione che dica r_ij = 1 => porto oggetto i come j-esimo => entrambi gli encoding, uno mi serve per Hamilton e uno mi serve per obj_func
        # # TODO: is this optimizable?? Look online for encoding of Hamiltonian cycles (n.b. NOT solutions to the general HCP, Hamiltonian Cycle Problem)
        # for j in range(n+1):
        #     for k in range(n+1):
        #         if k == j:
        #             solver.add(Not(z[i][k][k])) # Diagonal has to be zero: we say that cannot reach a node from itself
        #         # If j->k is in route, then it's also reachable k from j 
        #         solver.add(Implies(r[i][j][k], z[i][j][k])) # TODO: this is a useless constraint or an implicit (=> useful) one? Check performance without

        #         # If we can say that k it's reachable from j, it means that either 
        #         # (i): j->k is in the route
        #         # or (ii): exists (Or) a node h such that h is reachable from j in route and h->k is in route
        #         solver.add(Implies(z[i][j][k], 
        #                            Or(
        #             r[i][j][k], 
        #             Or(
        #             [And(z[i][j][h], r[i][h][k]) for h in range(n+1)]
        #             ))))  
        # # Goal: want that from Origin (row n) we can reach all the delivery points that courier i has to do (i.e. a[i])
        # solver.add(equal(a[i], z[i][n][:-1], n))
        ###

        ### Scaio's idea:
        # for k in range(n):
        #     solver.add(Not(all_false([r[i][j][h] for j in range(n, k, -1) for h in range(k+1)])))
        ###

        ### Edo's 2nd idea:
        # t_jk = 1 iff object j is delivered as k-th element (i.e. at TIME=k, use of time as order)
        # TODO 3: move t definition here from above
        for j in range(n): 
            # assignments implications
            solver.add(Implies(a[i][j], exactly_one_seq(t[i][j], f'sometime_{j}_by_courier_{i}')))  # a_ij (j assigned to i) -> [t_ijk = True, for some k] (i delivers j at some time k)
            solver.add(Implies(Not(a[i][j]), all_false(t[i][j])))                                   # Not(a_ij) (j not assigned to i) -> [t_ijk = False, forall k] (i never delivers j)

            # can't deliver two items at the same time
            solver.add(at_most_one_seq([t[i][k][j] for k in range(n)], f"no_contemporary_delivery_{i}_{j}"))

            # 1st trip
            solver.add(Implies(r[i][n][j], t[i][j][0])) # r_inj (i rides n (origin) --> j) => t_ij0 (i delivers j as 1st item)

            # successive trips
            for h in range(1, n):
            # TODO: Domanda: devo aggiungerlo? secondo e basta la condizione sufficiente ma magari questa aiuta poichè implicit constraint!!
            #     for k in range(n):
            #         solver.add(Implies(And(r[i][j][k], t[i][j][h-1]), t[i][k][h]))  # r_i,j,k AND t_i,j,h-1 (i goes from j to k AND he delivered j at time h-1) => t_i,k,h (k delivered at time h)
                
                solver.add(Implies(t[i][j][h], Or([And(r[i][k][j], t[i][k][h-1]) for k in range(n)])))       # if t_i,k,h (k delivered at time h) => [r_i,j,k AND t_i,j,h-1. for some j] (i came from some j to k AND he delivered that j at time h-1)
        ###
        

    # Optimization search

    # TODO 1: move down
    distance_upper_bound = sum([max(D[i]) for i in range(n+1)])     # TODO: maybe we can find a tighter upper bound on obj func
    dist_digits = num_bits(distance_upper_bound) 

    # N.B. when flattening a matrix of Z3 Variables, the resulting elements of the list have the same reference as the ones from the matrix! Important for r
    # flatten r and D
    flat_r = [flatten(r[i]) for i in range(m)]
    flat_D = flatten(D)
    # convert flat_D to binary
    D_digits = num_bits(max(e for e in flat_D))
    flat_D_bin = [int_to_bin(e, dist_digits) for e in flat_D]  #TODO 2: change to D_digits

    # distances[i] := binary representation of the distance travelled by courier i
    # TODO 1: move here
    solver.push()   # when backtracking, I want to use less digits! That's why I push the frame now   
    distances = [[Bool(f"dist_bin_{i}_{k}") for k in range(dist_digits)] for i in range(m)]
    # N.B. could either (1) find an upper bound to the objective value and use it immediately as a constraint (more information immediately, 
    # but is this allowed? can I use any algo to do so?)
    # or (2) run the model with no bound on the objective function the first time, then use the first value found as the initial bound
    # Question: can I do (1) and find a bound that is usually better than the bound found at (2)? If so, this would mean more info + better bound from the start!!
    
    # definition of distances using constraints
    solver.add(distances_def_constraint(distances, flat_r, flat_D_bin)) # TODO: define a function in order to call it many times on distances w/ different length! i.e. different number of digits
                                                                        # i.e. implement sum over numbers with different number of bits
    
    model = None
    obj_value = None
    while solver.check() == z3.sat:
        model = solver.model()
        obj_value = obj_function(model, distances)    # TODO: check that dist_digits of the modes actually contains the distance travelled by courier (check using r and D)
        print(f"This model obtained objective value: {obj_value}")

        if obj_value <= 1:
            break
        
        obj_value_digits = num_bits(obj_value - 1)  # <= obj_value - 1  <==>  < obj_value #TODO: could not do this and pass immediately the evaluation of distance with maximum value, smaller overhead...
                                                                                          # BUT then I couldn't print the line above!
        obj_value_bin = int_to_bin(obj_value - 1, dist_digits)

        solver.pop()
        solver.push()
        # TODO: IMPORTANT - check if recreating the distances variables every time actually slows down the solver (probably in case of solver that uses explanations/heuristics!)
        # even tho they have less digits every time
        distances = [[Bool(f"dist_bin_{i}_{k}_") for k in range(dist_digits)] for i in range(m)]        
        solver.add(distances_def_constraint(distances, flat_r, flat_D_bin)) # TODO: problemi: flat_D_bin[i] e distances non avranno gli stessi bits! distances molti di piu -> easy fix: implementa sum con generico numero di digits (Q: num(a) <= num(b) posso assumerlo??)
        solver.add(AllLessEq_bin(distances, obj_value_bin, dist_digits)) # TODO: this time, LInearInteger called inside AllLess must be <, not <= !!! Pass it a parameter to use leq or (to implement) lt (less than)
    ### then remove this below 
    # print(solver.check())
    # final_model = solver.model()
    ### up to this point
    # TODO: re-do thing above again but for BINARY_SEARCH, not LINEAR_SEARCH, then add parameter to the input of the model to choose between which one to use

    if model is None:
        return "UNSAT"
    
    ### Add by Scaio

    # print(max_dist(final_model, r, a, D, m, n))
    R = tensor(model, r)
    T = tensor(model, t)
    display_matrices(R, 'r')
    display_matrices(T, 't')
    display_matrices(distances, 'distances')
    print(f"The tensor contains all hamiltonian cycles: {check_all_hamiltonian(R)}")
    
    ###
    return (R, obj_value)


### Testing Model 1

In [123]:
m = 2
n = 2
l = [5, 5]
s = [1, 1]
D = [[0, 3, 3],
     [3, 0, 4],
     [3, 4, 0]]

displayMCP(multiple_couriers_planning_1(m, n, l, s, D))

digits = 3
[And(Or(a_0_0, a_1_0),
     And(Or(Not(a_0_0), s_0_assignment_0),
         Or(Not(s_0_assignment_0), Not(a_1_0)))),
 And(Or(a_0_1, a_1_1),
     And(Or(Not(a_0_1), s_0_assignment_1),
         Or(Not(s_0_assignment_1), Not(a_1_1)))),
 And(And(And(Implies(a_0_0,
                     And(d_0_0_load_capacity_courier_0 ==
                         False,
                         d_0_1_load_capacity_courier_0 ==
                         False,
                         d_0_2_load_capacity_courier_0 ==
                         True)),
             Implies(Not(a_0_0),
                     And(Not(d_0_0_load_capacity_courier_0),
                         Not(d_0_1_load_capacity_courier_0),
                         Not(d_0_2_load_capacity_courier_0)))),
         And(Implies(a_0_1,
                     And(LIsum_result_2_load_capacity_courier_0 ==
                         Or(And(d_0_2_load_capacity_courier_0,
                                True,
                                False),
   

In [120]:
m = 1
n = 3
l = [15]
s = [3, 2, 4]
D = [[0, 3, 3, 2],
     [3, 0, 4, 3],
     [3, 4, 0, 3],
     [2, 3, 3, 0]]

displayMCP(multiple_couriers_planning_1(m, n, l, s, D))

digits = 4
This model obtained objective value: 12
r_0 matrix:
[False, False, True, False]
[False, False, False, True]
[False, True, False, False]
[True, False, False, False]

t_0 matrix:
[True, False, False]
[False, False, True]
[False, True, False]

distances_0 matrix:
dist_bin_0_0_
dist_bin_0_1_
dist_bin_0_2_
dist_bin_0_3_

The tensor contains all hamiltonian cycles: True
------Objective value: 12------
-------------Routes------------
Origin --> 0 --> 2 --> 1 --> Origin


In [105]:
m = 3
n = 7
l = [15, 10, 7]
s = [1, 1, 1, 1, 1, 1, 1]
D = [[0, 3, 3, 6, 5, 6, 6, 2],
     [3, 0, 4, 3, 4, 7, 7, 3],
     [3, 4, 0, 7, 6, 3, 5, 3],
     [6, 3, 7, 0, 3, 6, 6, 4],
     [5, 4, 6, 3, 0, 3, 3, 3],
     [6, 7, 3, 6, 3, 0, 2, 4],
     [6, 7, 5, 6, 3, 2, 0, 4],
     [2, 3, 3, 4, 3, 4, 4, 0]]

displayMCP(multiple_couriers_planning_1(m, n, l, s, D))

UNSAT


In [104]:
m = 1
n = 3
l = [15]
s = [3, 2, 4]
D = [[0, 1, 1, 1],
     [1, 0, 1, 1],
     [1, 1, 0, 1],
     [1, 1, 1, 0]]

displayMCP(multiple_couriers_planning_1(m, n, l, s, D))

This model obtained objective value: 4
r_0 matrix:
[False, False, True, False]
[False, False, False, True]
[False, True, False, False]
[True, False, False, False]

t_0 matrix:
[True, False, False]
[False, False, True]
[False, True, False]

distances_0 matrix:
dist_bin_0_0
dist_bin_0_1
dist_bin_0_2

The tensor contains all hamiltonian cycles: True
------Objective value: 4------
-------------Routes------------
Origin --> 0 --> 2 --> 1 --> Origin


In [19]:
for k in range(3):
    print([(j, h) for j in range(n, k, -1) for h in range(k+1)])

[(3, 0), (2, 0), (1, 0)]
[(3, 0), (3, 1), (2, 0), (2, 1)]
[(3, 0), (3, 1), (3, 2)]


In [20]:
x = [Bool(f'c{i}') for i in range(4)]
z = [e for e in x]

solver = Solver()
solver.add(And(x))
solver.check()
m = solver.model()

print([m.evaluate(x[i]) for i in range(4)])
print([m.evaluate(z[i]) for i in range(4)])
print([id(x[i]) for i in range(4)])
print([id(z[i]) for i in range(4)])
print(id(x), id(z))

[True, True, True, True]
[True, True, True, True]
[2320490873416, 2320491308296, 2320491310984, 2320491266120]
[2320490873416, 2320491308296, 2320491310984, 2320491266120]
2320435931976 2320435932808


In [21]:
numbers = [[j*2 + i for j in range(1000)] for i in range(1000)]

In [40]:
print(num_bits)

0
